# tem Beneficios a entrega contínua?

## Revisar literatura para encontrar melhores perguntas de pesquisa

## Analisar dataset para estabelecer quais projetos tem entrega contínua



In [ ]:
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId
from datetime import timedelta
import numpy as np
import datetime
import re
client = MongoClient()
db = client.smartshark_2_2

### 1) Tem integração contínua - OK

In [ ]:
vcs_systems = pd.DataFrame(list(db.vcs_system.find()))
travisBuilds = pd.DataFrame(list(db.travis_build.find()))
projetosComBuilds = travisBuilds['vcs_system_id'].unique()
print(vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna().count())
vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()

### 2) Pegar as tags do projetos de integração contínua? - OK

In [ ]:
tags = pd.DataFrame(list(db.tag.find()))
idsVcsComBuilds = vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()._id

In [ ]:
tagsDosProjComBuild = tags.query('vcs_system_id.isin(@idsVcsComBuilds)').drop_duplicates().dropna()

In [ ]:
tagsDosProjComBuild

### 3) Determinei oq vou considerar Tag de Release - OK
##### Tags de release não tem no nome os termos 'RC', 'candidate', 'test', 'beta', 'alfa', 'unreleased', snapshot, siteupdate, freeze, sandbox, migration

In [ ]:
tagsDeRelase = tagsDosProjComBuild[
    tagsDosProjComBuild['name']
    .str.contains('rc|RC|candidate|CANDIDATE|test|TEST|beta|BETA|alpha|ALPHA|unreleased|UNRELEASED|snapshot|SNAPSHOT|siteupdate|SITEUPDATE|freeze|FREEZE|sandbox|SANDBOX|migration|MIGRATION|dev|DEV')==False].drop_duplicates().dropna()

In [ ]:
tagsDeRelase.name.to_csv('tags.csv', index=False)

In [ ]:
tagsDeRelase

### 4) Filtrar builds que tem commits de tag de release - OK

In [ ]:
commitsDeRelease = tagsDeRelase.commit_id.unique()
buildsIntegrados = travisBuilds.query('commit_id.isin(@commitsDeRelease)')
stateIndesejados = ['failed', 'canceled']
BuildsValidos = buildsIntegrados.query('~state.isin(@stateIndesejados)')

In [ ]:
BuildsValidos.state.value_counts()

### 5) Analisar quais projetos tem builds de entrega ok 

In [ ]:
vcsComEntrega = BuildsValidos.vcs_system_id.unique()
print(vcs_systems.query('_id.isin(@vcsComEntrega)').drop_duplicates().dropna().count())
vcs_systems.query('_id.isin(@vcsComEntrega)').drop_duplicates().dropna()

### 6) Desses projetos quais tem uma quantidade relevante de entregas que permita a nossa análise (>3 Builds com integração) - OK


In [ ]:
BuildsValidosPorCommit = BuildsValidos[['_id', 'vcs_system_id', 'commit_id', 'number', 'state', 'started_at', 'finished_at']].groupby(['commit_id'], as_index=False, sort=False).max()

In [ ]:
vcsComCD = BuildsValidosPorCommit[['vcs_system_id','_id']].groupby(['vcs_system_id'], as_index=False).agg('count').sort_values(by=['_id'], ascending=False)
vcsComCD = vcsComCD.query('_id > 2')
print(vcsComCD.count())
vcsComCD

## Validação de continuidade de uso de CD (em paralelo uma validação manual)

Quantos Releases tem a partir da adoção  X Quantos Builds com commits de Release a partir da data da adoção de CD 

# Fim caracterização

### 7) Percorrer os projetos e coletar seus dados

In [ ]:
vcsList= []
for index, row in vcsComCD.iterrows():
    vcsList.append(row.vcs_system_id)
vcsList

Montar a primeira tabela, que para cada vcs_system_id, busca project_id, issues_system_id, data da adoção de CD, primeira build de CD, quantas builds antes e depois do CD, campo de análise manual se tem a CD suspeita

In [ ]:
issue_system = pd.DataFrame(list(db.issue_system.find()))
projects = pd.DataFrame(list(db.project.find()))
listaDosProjetosComCD = [] 
issues_system_ids = []
for vcs in vcsList:
    # Buscar informações sobre o projeto
    project_id = vcs_systems.query('_id == @vcs').iloc[0,2] 

    #nome do projeto
    name = projects.query('_id == @project_id').iloc[0,1]     

    # Informações sobre o issue system
    issues_system_id = issue_system.query('project_id == @project_id').iloc[0,0]
    issues_system_ids.append(issues_system_id)   

    # Pegar o commit da build considerada como a build de adoção (desconsiderando o primeiro)
    commit_id_DaBuilld = BuildsValidosPorCommit.query('vcs_system_id == @vcs').sort_values(by=['started_at']).iloc[1,0]
    

    # Pegar a release com o commit de build de adoção
    id_tag_Adocao_CD = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,0]
    nome_da_releasa = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,1]
    data_de_adocao = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,6]

    listaDosProjetosComCD.append([name , vcs,project_id, issues_system_id, commit_id_DaBuilld, id_tag_Adocao_CD, nome_da_releasa, data_de_adocao  ])

dfProjetosComCD = pd.DataFrame(listaDosProjetosComCD, columns=['name', 'vcs_system_id', 'project_id', 'issues_system_id', 'commit_id', 'tag_id','adoption_release', 'cd_adoption_date' ])
dfProjetosComCD

### 8) Buscar as issues dos projetos analisados para contabilizar as metricas

In [ ]:
issues = pd.DataFrame(list(db.issue.find()))
issuesProjetosCD = issues.query('issue_system_id.isin(@issues_system_ids)')

### 9) Contabilizar métricas


#### 1) Time to market (tempo entre releases)
Tempo entre releases (tempo medio)


Algoritmo Final:

In [ ]:
difference_per_project = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    tmp_date = dfProjetosComCD.iloc[i,7]
    ttm_tags_no_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date < @tmp_date ').sort_values(by='date')
    ttm_tags_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date >= @tmp_date ').sort_values(by='date')
    average_no_cd = (ttm_tags_no_cd.iloc[ttm_tags_no_cd.shape[0] - 1,6] - ttm_tags_no_cd.iloc[0,6])/(ttm_tags_no_cd.shape[0] - 1)
    average_cd = (ttm_tags_cd.iloc[ttm_tags_cd.shape[0] - 1,6] - ttm_tags_cd.iloc[0,6])/(ttm_tags_cd.shape[0] - 1)
    difference_per_project.append((tmp_name, tmp_vcs, average_no_cd - average_cd))


In [ ]:
df_tmt = pd.DataFrame(difference_per_project, columns=['name', 'vcs_system_id', 'noCD_CD'])
df_tmt

In [ ]:
temp_delta_zero = datetime.datetime.now() - datetime.datetime.now()
print('Projetos que diminuiram seu Time to Market: ', df_tmt.query('noCD_CD > @temp_delta_zero').count().iloc[0])
print('Projetos que aumentaram seu Time to Market: ', df_tmt.query('noCD_CD < @temp_delta_zero').count().iloc[0]) 


In [ ]:
difference_per_project = []
list_builds = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    tmp_adoption_date = dfProjetosComCD.iloc[i,7]
    tmp_df = tagsDeRelase.query('vcs_system_id == @tmp_vcs').sort_values(by='date')
    for j in range(tmp_df.shape[0]):
        tmp_release_date = tmp_df.iloc[j,6]
        if j != 0:
            tmp_date_diference = tmp_release_date - tmp_df.iloc[j-1,6]
            cd_flag = tmp_release_date >= tmp_adoption_date
            release_name = tmp_df.iloc[j,1]
            list_builds.append([tmp_name,tmp_vcs, release_name, tmp_release_date, tmp_date_diference, cd_flag ])                    
df_tmt_builds = pd.DataFrame(list_builds, columns=['name','vcs_system_id', 'release_name','release_date', 'deltatime_last_release', 'isCD'])    

In [ ]:
mean_release_tax_no_cd = []
mean_release_tax_cd = []
median_release_tax_no_cd = []
median_release_tax_cd = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    mean_release_tax_no_cd.append(df_tmt_builds.query('vcs_system_id == @tmp_vcs and isCD == False').deltatime_last_release.mean())
    mean_release_tax_cd.append(df_tmt_builds.query('vcs_system_id == @tmp_vcs and isCD == True').deltatime_last_release.mean())
    median_release_tax_no_cd.append(df_tmt_builds.query('vcs_system_id == @tmp_vcs and isCD == False').deltatime_last_release.median())
    median_release_tax_cd.append(df_tmt_builds.query('vcs_system_id == @tmp_vcs and isCD == True').deltatime_last_release.median())
dfProjetosComCD['mean_release_tax_no_cd'] = mean_release_tax_no_cd
dfProjetosComCD['mean_release_tax_cd'] = mean_release_tax_cd
dfProjetosComCD['median_release_tax_no_cd'] = median_release_tax_no_cd
dfProjetosComCD['median_release_tax_cd'] = median_release_tax_cd

In [ ]:
print('Projetos com intervalo media entre releases maior após CD:', dfProjetosComCD.query('mean_release_tax_no_cd < mean_release_tax_cd').count().iloc[0] )
print('Projetos com intervalo media entre releases menor após CD:', dfProjetosComCD.query('mean_release_tax_no_cd >= mean_release_tax_cd').count().iloc[0] )
print('Projetos com mediana de intervalo entre releases maior após CD:', dfProjetosComCD.query('median_release_tax_no_cd < median_release_tax_cd').count().iloc[0] )
print('Projetos com mediana de intervalo entre releases menor após CD:', dfProjetosComCD.query('median_release_tax_no_cd >= median_release_tax_cd').count().iloc[0] )

#### 2) Quantas issues endereçadas (fix_version) e fechadas por release Issues com status 'Closed' e com fix_version direcionado a um versão específica - Não faz sentido - 

In [ ]:
listIssuesFixVersion = []
for i in range(dfProjetosComCD.shape[0]):
    FVersionsDict = {}
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    tmp_name = dfProjetosComCD.iloc[i,0]
    for i in issues.query('issue_system_id == @tmp_issue_sistem & status == \'Closed\'').fix_versions:
        if isinstance(i, list): 
            for j in i:
                if j not in FVersionsDict:
                    FVersionsDict[j] = 1
                else:
                    FVersionsDict[j] += 1    
    for key in FVersionsDict:
        listIssuesFixVersion.append([tmp_name, tmp_issue_sistem, key, FVersionsDict[key] ])
df_Issues_fix_Version = pd.DataFrame(listIssuesFixVersion, columns=['name', 'issue_sistem_id', 'version', 'issues'])
df_Issues_fix_Version

Requer validação manual pra fazer isso, mas seria legal esses boxplots

In [ ]:
tmp_issue_sistem = ObjectId('5bc052df762a157bad4494ca')
tmp_version = '3.6'
df_Issues_fix_Version.query('issue_sistem_id == @tmp_issue_sistem ').sort_values(by='version')

In [ ]:
#tmp_issue_sistem = ObjectId('5d7778c99f03373d37512b7e')
#tmp_version = '4.5.4'
df_Issues_fix_Version.query('issue_sistem_id == @tmp_issue_sistem & version < @tmp_version ').sort_values(by='version').issues.describe()

In [ ]:
df_Issues_fix_Version.query('issue_sistem_id == @tmp_issue_sistem & version >= @tmp_version ').sort_values(by='version').issues.describe()

#### 3) Quantas issues fechadas por periodo (e prioridade) Issues com status 'Closed' e com updated_at entre os periodos das relases

taxa de fechamento de issue dentro do espaço da release

tempo medio (mediana) de fechamento de issue antes e depois de CD

Algoritmo Final:

In [ ]:
averages_by_project = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    tmp_date = dfProjetosComCD.iloc[i,7]
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    ttm_tags_no_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date < @tmp_date ').sort_values(by='date')
    ttm_tags_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date >= @tmp_date ').sort_values(by='date')
    
    average_no_cd = 0
    for j in range(0,ttm_tags_no_cd.shape[0] - 1):
        data_release_anterior = ttm_tags_no_cd.iloc[j,6]
        data_release_da_vez = ttm_tags_no_cd.iloc[j+1,6]
        average_no_cd += issues.query('issue_system_id == @tmp_issue_sistem & status == \'Closed\' & updated_at > @data_release_anterior  & updated_at <= @data_release_da_vez').shape[0]
    average_no_cd = average_no_cd/(ttm_tags_no_cd.shape[0] - 1)
    
    average_cd = 0
    for j in range(0,ttm_tags_cd.shape[0] - 1):
        data_release_anterior = ttm_tags_cd.iloc[j,6]
        data_release_da_vez = ttm_tags_cd.iloc[j+1,6]
        average_cd += issues.query('issue_system_id == @tmp_issue_sistem & status == \'Closed\' & updated_at > @data_release_anterior  & updated_at <= @data_release_da_vez').shape[0]
    average_cd = average_cd/(ttm_tags_cd.shape[0] - 1)

    averages_by_project.append((tmp_name, tmp_vcs, average_no_cd, average_cd))

df_media_issues_periodo = pd.DataFrame(averages_by_project, columns=['name', 'vcs_system_id', 'no_CD', 'CD'])
    

In [ ]:
issues.query('issue_system_id == @tmp_issue_sistem & status == \'Closed\' ').issue_type.value_counts()

In [ ]:
print('Projetos com media de issues fechadas maior após CD:', df_media_issues_periodo.query('no_CD < CD').count().iloc[0] )
print('Projetos com media de issues fechadas menor após CD:', df_media_issues_periodo.query('no_CD > CD').count().iloc[0] )

#### 4) Bugs reportados por release (affects_versions) -- Mais precisa -- Issues do tipo 'Bug' com affects_versions direcionado a um versão específica

In [ ]:
listIssuesAffectsVersions = []
for i in range(dfProjetosComCD.shape[0]):
    AfVersionsDict = {}
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    tmp_name = dfProjetosComCD.iloc[i,0]
    for i in issues.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\'').affects_versions:
        if isinstance(i, list): 
            for j in i:
                if j not in AfVersionsDict:
                    AfVersionsDict[j] = 1
                else:
                    AfVersionsDict[j] += 1    
    for key in AfVersionsDict:
        tmp_key = re.sub("[^\d\.]", "", key)
        listIssuesAffectsVersions.append([tmp_name, tmp_issue_sistem, tmp_key, AfVersionsDict[key] ])
df_Issues_Affects_versions = pd.DataFrame(listIssuesAffectsVersions, columns=['name', 'issue_sistem_id', 'version', 'bugs'])
df_Issues_Affects_versions

Apos validação manual da versão de adoção no issue sistem

In [ ]:
adoption_version_issues_system = ['1.0.0', '1.5.4.1', '2.5','1.1.0','1.7.2','1.14','4.4.11','2.7.1', '4.2.0', '4.5.4', '3.6', '2.3.0', '0.4', '2.2', '0.53.0', '4.0.5', '0.9.0', '6.2', '1.8.2', '2.0.11', '2.1']
dfProjetosComCD['adoption_version_issues_system'] = adoption_version_issues_system

In [ ]:
listmedia_no_cd = []
listmedia_cd = []
listmediana_no_cd = []
listmediana_cd = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    tmp_version = dfProjetosComCD.iloc[i,12]
    listmedia_no_cd.append(df_Issues_Affects_versions.query('issue_sistem_id == @tmp_issue_sistem & version < @tmp_version').bugs.mean())
    listmedia_cd.append(df_Issues_Affects_versions.query('issue_sistem_id == @tmp_issue_sistem & version >= @tmp_version').bugs.mean())
    listmediana_no_cd.append(df_Issues_Affects_versions.query('issue_sistem_id == @tmp_issue_sistem & version < @tmp_version').bugs.median())
    listmediana_cd.append(df_Issues_Affects_versions.query('issue_sistem_id == @tmp_issue_sistem & version >= @tmp_version').bugs.median())
dfProjetosComCD['afv_mean_open_bugs_no_cd'] = listmedia_no_cd
dfProjetosComCD['afv_mean_open_bugs_cd'] = listmedia_cd
dfProjetosComCD['afv_median_open_bugs_no_cd'] = listmediana_no_cd
dfProjetosComCD['afv_median_open_bugs_cd'] = listmediana_cd

In [ ]:
print('Projetos com media de Bugs reportados explicitamente maior após CD:', dfProjetosComCD.query('afv_mean_open_bugs_no_cd < afv_mean_open_bugs_cd').count().iloc[0])
print('Projetos com media de Bugs reportados explicitamente menor após CD:', dfProjetosComCD.query('afv_mean_open_bugs_no_cd >= afv_mean_open_bugs_cd').count().iloc[0])
print('Projetos com mediana de Bugs reportados explicitamente maior após CD:', dfProjetosComCD.query('afv_median_open_bugs_no_cd < afv_median_open_bugs_cd').count().iloc[0] )
print('Projetos com mediana de Bugs reportados explicitamente menor após CD:', dfProjetosComCD.query('afv_median_open_bugs_no_cd >= afv_median_open_bugs_cd').count().iloc[0] )

In [ ]:
dfProjetosComCD[['name','adoption_release', 'afv_mean_open_bugs_no_cd', 'afv_mean_open_bugs_cd' ,'afv_median_open_bugs_no_cd','afv_median_open_bugs_cd']]

#### 5) Bugs reportados (abertos) no periodo por release -- Mais factual -- Issues do tipo 'Bug' e com created_at entre os periodos das relases

In [ ]:
bugs_averages_by_project = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    tmp_date = dfProjetosComCD.iloc[i,7]
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    ttm_tags_no_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date < @tmp_date ').sort_values(by='date')
    ttm_tags_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date >= @tmp_date ').sort_values(by='date')
    
    bugs_average_no_cd = 0
    bugs_median_no_cd = 0
    aux_array = np.zeros((ttm_tags_no_cd.shape[0] - 1))
    for j in range(0,ttm_tags_no_cd.shape[0] - 1):
        data_release_anterior = ttm_tags_no_cd.iloc[j,6]
        data_release_da_vez = ttm_tags_no_cd.iloc[j+1,6]
        aux_array[j] = (issues.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' & created_at > @data_release_anterior  & created_at <= @data_release_da_vez').shape[0])
    bugs_average_no_cd = np.mean(aux_array)
    bugs_median_no_cd = np.median(aux_array)
    
    bugs_average_cd = 0
    bugs_median_cd = 0
    aux_array = np.zeros((ttm_tags_cd.shape[0] - 1))
    for j in range(0,ttm_tags_cd.shape[0] - 1):
        data_release_anterior = ttm_tags_cd.iloc[j,6]
        data_release_da_vez = ttm_tags_cd.iloc[j+1,6]
        aux_array[j] = (issues.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' & created_at > @data_release_anterior  & created_at <= @data_release_da_vez').shape[0])
    bugs_average_cd = np.mean(aux_array)
    bugs_median_cd = np.median(aux_array)

    bugs_averages_by_project.append((tmp_name, tmp_vcs, bugs_average_no_cd, bugs_average_cd, bugs_median_no_cd, bugs_median_cd))
df_media_bugs_abertos_entre_releases = pd.DataFrame(bugs_averages_by_project, columns=['name', 'vcs_system_id', 'no_CD_mean', 'CD_mean', 'no_CD_median', 'CD_median'])
df_media_bugs_abertos_entre_releases[['name','no_CD_mean', 'CD_mean', 'no_CD_median', 'CD_median' ]]

In [ ]:
df_media_bugs_abertos_entre_releases[['name','no_CD_mean', 'CD_mean', 'no_CD_median', 'CD_median' ]]

In [ ]:
print('Projetos com media de Bugs reportados maior após CD:', df_media_bugs_abertos_entre_releases.query('no_CD_mean < CD_mean').count().iloc[0] )
print('Projetos com media de Bugs reportados menor após CD:', df_media_bugs_abertos_entre_releases.query('no_CD_mean >= CD_mean').count().iloc[0] )
print('Projetos com mediana de Bugs reportados maior após CD:', df_media_bugs_abertos_entre_releases.query('no_CD_median < CD_median').count().iloc[0] )
print('Projetos com mediana de Bugs reportados menor após CD:', df_media_bugs_abertos_entre_releases.query('no_CD_median >= CD_median').count().iloc[0] )

In [ ]:
listIssuesAffectsVersions = []
bugsWithNoAffects = []
for i in range(dfProjetosComCD.shape[0]):
    AfVersionsDict = {}
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    tmp_name = dfProjetosComCD.iloc[i,0]
    data = issues.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\'')
    for i in range(data.shape[0]):
        if isinstance(data.iloc[i].affects_versions, list): 
            for j in data.iloc[i].affects_versions:
                if j not in AfVersionsDict:
                    AfVersionsDict[j] = 1
                else:
                    AfVersionsDict[j] += 1
        else:
            bugsWithNoAffects.append(data.iloc[i])
    for key in AfVersionsDict:
        tmp_key = re.sub("[^\d\.]", "", key)
        listIssuesAffectsVersions.append([tmp_name, tmp_issue_sistem, tmp_key, AfVersionsDict[key] ])
df_Issues_Affects_versions = pd.DataFrame(listIssuesAffectsVersions, columns=['name', 'issue_sistem_id', 'version', 'bugs'])
df_Issues_Affects_versions

In [ ]:
bugsWithNoAffects = pd.DataFrame(bugsWithNoAffects)

In [ ]:
bugs_averages_by_project_no_affects = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    tmp_date = dfProjetosComCD.iloc[i,7]
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    ttm_tags_no_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date < @tmp_date ').sort_values(by='date')
    ttm_tags_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date >= @tmp_date ').sort_values(by='date')
    
    bugs_average_no_cd = 0
    bugs_median_no_cd = 0
    aux_array = np.zeros((ttm_tags_no_cd.shape[0] - 1))
    for j in range(0,ttm_tags_no_cd.shape[0] - 1):
        data_release_anterior = ttm_tags_no_cd.iloc[j,6]
        data_release_da_vez = ttm_tags_no_cd.iloc[j+1,6]
        aux_array[j] = (bugsWithNoAffects.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' & created_at > @data_release_anterior  & created_at <= @data_release_da_vez').shape[0])
    bugs_average_no_cd = np.mean(aux_array)
    bugs_median_no_cd = np.median(aux_array)
    
    bugs_average_cd = 0
    bugs_median_cd = 0
    aux_array = np.zeros((ttm_tags_cd.shape[0] - 1))
    for j in range(0,ttm_tags_cd.shape[0] - 1):
        data_release_anterior = ttm_tags_cd.iloc[j,6]
        data_release_da_vez = ttm_tags_cd.iloc[j+1,6]
        aux_array[j] = (bugsWithNoAffects.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' & created_at > @data_release_anterior  & created_at <= @data_release_da_vez').shape[0])
    bugs_average_cd = np.mean(aux_array)
    bugs_median_cd = np.median(aux_array)

    bugs_averages_by_project_no_affects.append((tmp_name, tmp_vcs, bugs_average_no_cd, bugs_average_cd, bugs_median_no_cd, bugs_median_cd))
df_media_bugs_abertos_entre_releases_no_affects = pd.DataFrame(bugs_averages_by_project_no_affects, columns=['name', 'vcs_system_id', 'no_CD_mean', 'CD_mean', 'no_CD_median', 'CD_median'])
df_media_bugs_abertos_entre_releases_no_affects


In [ ]:
test = dfProjetosComCD[['name', 'vcs_system_id', 'afv_mean_open_bugs_no_cd', 'afv_mean_open_bugs_cd', 'afv_median_open_bugs_no_cd', 'afv_median_open_bugs_cd']]
test

In [ ]:
for i in range(test.shape[0]):
    test.iloc[i].afv_mean_open_bugs_no_cd += df_media_bugs_abertos_entre_releases_no_affects.iloc[i].no_CD_mean
    test.iloc[i].afv_mean_open_bugs_cd += df_media_bugs_abertos_entre_releases_no_affects.iloc[i].CD_mean
    test.iloc[i].afv_median_open_bugs_no_cd += df_media_bugs_abertos_entre_releases_no_affects.iloc[i].no_CD_median
    test.iloc[i].afv_median_open_bugs_cd += df_media_bugs_abertos_entre_releases_no_affects.iloc[i].no_CD_median
test